In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
import pandas as pd

In [2]:
text= "hello world this is retard number 1 . hello cunt you are a retard. hello stupid fuck you should kill yoruself . hello idiot you are so dumb no one wants to hire you . maybe you should stop being so dumb. You were born to kill yourself . You have nothing good going for you why are you still alive. Nobody likes you , you have no friends , no girl friends , you hate your job. why are you still alive . There is no point in you existing. So what are you gonna do? What are you gonna do about it ? This country hates you , people look at you with contempt , they want you gone . Why are you still here pajeet. I know this place is hell for me . I know no one likes me and most people hate me, I wished people were nice to me , i wish people would wouldnt look at me like i am a criminal. Its not my fault i was born brown. I have wanted to give up long ago , i wanted to go home ever since i came here. But i am no quitter. People who hate me for no reason i hope they dont have to face this upon themselves because that will make them realize how hard it is to be me.  Today i built Recurrent neural network by myself it feels pretty damn good. Maybe i should start building more and more ML models that would make me better at something. Today i learned about the dimensions of transformers. This week i plan to create Transformers from scratch including neural networks. By next week i should have implemented language translation , text summarization ."

In [3]:
f=open("incel.txt","r" ,encoding='cp932', errors='ignore')

In [4]:
dat=f.read()

In [5]:
#remove comment to train on text instead 
#text=dat

In [6]:
tokenizer=AutoTokenizer.from_pretrained("bert-base-uncased")
#tokenizes based on bert-based
tokens=tokenizer.tokenize(text) 
#converts tokens to integers
input_ids=tokenizer.convert_tokens_to_ids(tokens)    
#vocab size of tokenizer used 
vocab_size=tokenizer.vocab_size      

In [7]:
len(text)

1238

In [8]:
len(tokens)

290

**Testing out tokenizer and converting it back**

In [9]:
ha=tokenizer.convert_ids_to_tokens(input_ids[:20])
print(ha)

['hello', 'world', 'this', 'is', 're', '##tar', '##d', 'number', '1', '.', 'hello', 'cu', '##nt', 'you', 'are', 'a', 're', '##tar', '##d', '.']


In [10]:
seq_len=6
x_train=[]
y_train=[]
data=[]
for i in range(len(input_ids)-seq_len):
    dat_x=input_ids[i:i+seq_len]
    dat_y=input_ids[i+seq_len]
    data.append((torch.tensor(dat_x),dat_y))
    
    

In [11]:
data

[(tensor([7592, 2088, 2023, 2003, 2128, 7559]), 2094),
 (tensor([2088, 2023, 2003, 2128, 7559, 2094]), 2193),
 (tensor([2023, 2003, 2128, 7559, 2094, 2193]), 1015),
 (tensor([2003, 2128, 7559, 2094, 2193, 1015]), 1012),
 (tensor([2128, 7559, 2094, 2193, 1015, 1012]), 7592),
 (tensor([7559, 2094, 2193, 1015, 1012, 7592]), 12731),
 (tensor([ 2094,  2193,  1015,  1012,  7592, 12731]), 3372),
 (tensor([ 2193,  1015,  1012,  7592, 12731,  3372]), 2017),
 (tensor([ 1015,  1012,  7592, 12731,  3372,  2017]), 2024),
 (tensor([ 1012,  7592, 12731,  3372,  2017,  2024]), 1037),
 (tensor([ 7592, 12731,  3372,  2017,  2024,  1037]), 2128),
 (tensor([12731,  3372,  2017,  2024,  1037,  2128]), 7559),
 (tensor([3372, 2017, 2024, 1037, 2128, 7559]), 2094),
 (tensor([2017, 2024, 1037, 2128, 7559, 2094]), 1012),
 (tensor([2024, 1037, 2128, 7559, 2094, 1012]), 7592),
 (tensor([1037, 2128, 7559, 2094, 1012, 7592]), 5236),
 (tensor([2128, 7559, 2094, 1012, 7592, 5236]), 6616),
 (tensor([7559, 2094, 1012, 

In [12]:
dataloader = DataLoader(data, batch_size=30, shuffle=False)

In [13]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        super(RNNModel, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.learnm= nn.LSTM(embed_size, hidden_size, batch_first=True)
        #self.learnm=nn.RNN(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        x = self.embed(x)
        out,_ = self.learnm(x)
        out = self.fc(out[:, -1, :])
        return out 


In [14]:
embed_size = 20
hidden_size = 30
model = RNNModel(vocab_size, embed_size, hidden_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [15]:
lul=nn.Embedding(vocab_size,10)
kek=nn.Linear(10,40)

In [16]:
for inputs, target in dataloader:
    ot=lul(target)
    how=kek(ot)
    print(target.shape,ot.shape,how.shape)

torch.Size([30]) torch.Size([30, 10]) torch.Size([30, 40])
torch.Size([30]) torch.Size([30, 10]) torch.Size([30, 40])
torch.Size([30]) torch.Size([30, 10]) torch.Size([30, 40])
torch.Size([30]) torch.Size([30, 10]) torch.Size([30, 40])
torch.Size([30]) torch.Size([30, 10]) torch.Size([30, 40])
torch.Size([30]) torch.Size([30, 10]) torch.Size([30, 40])
torch.Size([30]) torch.Size([30, 10]) torch.Size([30, 40])
torch.Size([30]) torch.Size([30, 10]) torch.Size([30, 40])
torch.Size([30]) torch.Size([30, 10]) torch.Size([30, 40])
torch.Size([14]) torch.Size([14, 10]) torch.Size([14, 40])


In [17]:
epochs = 200
for epoch in range(epochs):
    for inputs, target in dataloader:
        optimizer.zero_grad()
        output=model(inputs)
        loss =criterion(output, target)
        #optimizer.step()
        loss.backward()
        optimizer.step()

    if epoch%10==0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')

Epoch 0, Loss: 10.1866
Epoch 10, Loss: 3.9841
Epoch 20, Loss: 3.1749
Epoch 30, Loss: 2.1118
Epoch 40, Loss: 1.1970
Epoch 50, Loss: 0.6856
Epoch 60, Loss: 0.5487
Epoch 70, Loss: 0.2662
Epoch 80, Loss: 0.1614
Epoch 90, Loss: 0.1066
Epoch 100, Loss: 0.0761
Epoch 110, Loss: 0.0952
Epoch 120, Loss: 0.0479
Epoch 130, Loss: 0.0363
Epoch 140, Loss: 0.0293
Epoch 150, Loss: 0.0246
Epoch 160, Loss: 0.0211
Epoch 170, Loss: 0.0184
Epoch 180, Loss: 0.0161
Epoch 190, Loss: 0.0142


In [20]:
howmanywords=150

In [21]:
def generate_text(model, start_text, next_words):
    model.eval()
    tokens = tokenizer.tokenize(start_text)
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    hidden = torch.zeros(1, 1, hidden_size)
    for _ in range(next_words):
        #even though input_ids are less than sequence length here, it works because we arent using fixed dimensions neural network.
        input_seq = torch.tensor(input_ids[-seq_len:], dtype=torch.long).unsqueeze(0)  # Ensure shape (1, seq_length)
        with torch.no_grad():
            output = model(input_seq)
        next_token_id = output.argmax().item()
        input_ids.append(next_token_id)
    return tokenizer.decode(input_ids)


print(generate_text(model, "how",howmanywords))

hownt why are i am a retard. maybe you are a retard. hello stupid fuck you should kill yoruself. hello idiot you are so dumb no one wants to hire you. maybe you should stop being so dumb. you were born to kill yourself. you have nothing good going for you why are you still alive. there is no point in you existing. so what are you gonna do? what are you gonna do about it? this country hates you, people look at you with contempt, they want you gone. why are you still here pajeet. i know this place is hell for me. i know no one likes me and most people hate me, i wished people were nice to me, i wish
